Imports


In [ ]:
import pandas as pd
import numpy as np
import json
import pickle
import math
import sys

#Task 1: Vocabulary Creation 

Load Train Data


In [ ]:
dir = '/content/drive/MyDrive/544/'
df = pd.read_csv(dir+'train',sep='\t',header=None,skip_blank_lines=True)
df_dev = pd.read_csv(dir+'dev',sep='\t',header=None,skip_blank_lines=True)
df_test = pd.read_csv(dir+'test',sep='\t',header=None,skip_blank_lines=True)

FileNotFoundError: ignored

In [ ]:
df_dev.shape

(131768, 3)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Replacing the values which occure less than threshold with special token '< unk >'

In [ ]:
thrsh = 3

df_voc = pd.DataFrame(df[1].value_counts()).reset_index()

df_voc['index'] = np.where(df_voc[1]<thrsh,'<unk>',df_voc['index'])

In [ ]:
df_voc = df_voc.groupby(df_voc['index']).sum()

Sorting with occurances

In [ ]:
df_voc = df_voc.sort_values(by=1,ascending=False).reset_index()

Save to txt file

In [ ]:
vocab = df_voc['index']

In [ ]:
df_voc.to_csv('vocab.txt',sep='\t',header = False,index=False)

Replacing values with < unk > in the train data

In [ ]:
voc = pd.DataFrame(df[1].value_counts()).reset_index().rename(columns={'index':1,1:3})

In [ ]:
df = pd.merge(df,voc,on=1,how='left')

In [ ]:
df[1] = np.where(df[3]<3,'<unk>',df[1])

#Task 2: Model Learning

Creating Transition Matrix

In [ ]:
pos_set = list(set(df[2]))
transition_dict = dict()

for i in pos_set:
  transition_dict['init',i]=0

for i in pos_set:
  for j in pos_set:
    transition_dict[i,j]=0

for i in df.values:
  if i[0]==1:
    transition_dict['init',i[2]]+=1

for i in range(len(df[2])-1):
  transition_dict[df[2][i],df[2][i+1]]+=1

In [ ]:
occur = df[2].value_counts()
occur['init'] = df[0].value_counts()[1]
for i,j in transition_dict.items():
  transition_dict[i] = j/occur[i[0]]

Creating Emission Matrix

In [ ]:
word_set = list(set(df[1]))
emission_dict = dict()
for i in pos_set:
  
  for j in word_set:
    emission_dict[i,j]=0
    
for i in df.values:
  emission_dict[(i[2],i[1])]+=1

In [ ]:
len(list(set([k[0] for k in emission_dict.keys()])))

45

In [ ]:
occur = df[2].value_counts()
for i,j in emission_dict.items():
  emission_dict[i] = j/occur[i[0]]

In [ ]:
hmm = dict()
hmm['transition'] = dict(map(lambda x: (str(x[0]),x[1]),transition_dict.items()))
hmm['emission'] = dict(map(lambda x: (str(x[0]),x[1]),emission_dict.items()))

In [ ]:
with open('hmm.json','w') as f:
  json.dump(hmm,f)

####To matrix

In [ ]:
keys = list(transition_dict.keys())
rows_labels = ['init']
cols_labels = pos_set
rows_labels.extend(cols_labels)
rows, cols = [rows_labels.index(k[0]) for k in keys], [cols_labels.index(k[1]) for k in keys]

In [ ]:
max(rows),max(cols)

(45, 44)

In [ ]:
transition_matrix = np.zeros((max(rows)+1, max(cols)+1))
for i, j in zip(rows, cols):
    transition_matrix[i, j] = transition_dict[(rows_labels[i],cols_labels[j])]

In [ ]:
keys = list(emission_dict.keys())
x_labels = cols_labels
y_labels = list(df_voc['index'])

In [ ]:
rows, cols = [x_labels.index(k[0]) for k in keys], [y_labels.index(k[1]) for k in keys]

In [ ]:
emission_matrix = np.zeros((max(rows)+1, max(cols)+1))

In [ ]:
for i, j in zip(rows, cols):
    emission_matrix[i, j] = emission_dict[(x_labels[i],y_labels[j])]

#Greedy Algorithm


In [ ]:
def greedy(states,data,transition,emission,vocab):
  tag_seq = []
  prev_state = None
  for i in data.values:
    word = i[1]
    if word not in vocab:
      word = '<unk>'
    if i[0]==1:
      prev_state = 'init'
    prev_state = states[np.argmax(list(map(lambda x: transition_dict[prev_state,x]*emission_dict[x,word],states)))]
    tag_seq.append(prev_state)
  return tag_seq

In [ ]:
states = pos_set
data = df_dev[[0,1]]
tag_seq = greedy(states,data,transition_dict,emission_dict,list(vocab))

In [ ]:
correct = 0
for i,j in zip(df_dev[2],tag_seq):
  if i == j:
    correct+=1
print(f'Accuracy: {round(correct*100/len(tag_seq),2)}%')

Accuracy: 92.99%


In [ ]:
data = df_test[[0,1]]
pred_seq_test = greedy(states,data,transition_dict,emission_dict,vocab)

In [ ]:
with open('greedy.out','w') as f:
  for i in zip(df_test.values,pred_seq_test):
    f.write(f'{i[0][0]}\t{i[0][1]}\t{i[1]}\n')

#Viterbi Algorithm 

In [ ]:
def viterbi_decoding(emission_matrix, transition_matrix, observation_sequence,pos_set,vocab):
    num_states = len(pos_set)
    num_observations = len(observation_sequence)
    viterbi = np.zeros((num_states, num_observations))
    backpointer = np.zeros((num_states, num_observations), dtype=int)
    o_0 = observation_sequence[0]
    if o_0 not in vocab:
        o_0 = '<unk>'
    for state in range(num_states):
        viterbi[state, 0] = transition_matrix['init', pos_set[state]] * emission_matrix[pos_set[state], o_0]
        backpointer[state, 0] = 0

    for t in range(1, num_observations):
        obs = observation_sequence[t]
        if observation_sequence[t] not in vocab:
            obs = '<unk>'
        
        for state in range(num_states):
            max_prob = -99999999
            max_state = -99999999
            for prev_state in range(num_states):
                prob = viterbi[prev_state, t-1] + math.log(sys.float_info.epsilon+transition_matrix[pos_set[prev_state], pos_set[state]]) + math.log(sys.float_info.epsilon+emission_matrix[pos_set[state], obs])
                if prob > max_prob:
                    max_prob = prob
                    max_state = prev_state
            viterbi[state, t] = max_prob
            backpointer[state, t] = max_state
    
    best_path = [0] * num_observations
    max_prob = -1
    final_state = -1
    for state in range(num_states):
        if viterbi[state, num_observations-1] > max_prob:
            max_prob = viterbi[state, num_observations-1]
            final_state = state
    
    best_path[num_observations-1] = final_state
    for t in range(num_observations-2, -1, -1):
        final_state = backpointer[final_state, t]
        best_path[t] = final_state
    
    return best_path


In [ ]:
path = viterbi_decoding(emission_dict,transition_dict, df_dev[1],pos_set,list(vocab))
pred_tags=[]
for i in path:
  pred_tags.append(pos_set[i])

In [ ]:
correct = 0
for i,j in zip(df_dev[2],pred_tags[1:]):
  if i == j:
    correct+=1
print(f'Accuracy: {correct*100/len(pred_tags)}%')

Accuracy: 94.40076498087548%


In [ ]:
path = viterbi_decoding(emission_dict,transition_dict, df_test[1],pos_set,list(vocab))
pred_tags=[]
for i in path:
  pred_tags.append(pos_set[i])

In [ ]:
with open('viterbi.out','w') as f:
  for i in zip(df_test.values,pred_tags):
    f.write(f'{i[0][0]}\t{i[0][1]}\t{i[1]}\n')